# S6E1 Doubly Censored Regression: XGBoost + Tobit + Bayes Optimal Prediction

### 📉 A Theoretical Approach to Bounded Targets

This notebook implements a **Doubly Censored Tobit Model** as a custom objective function for XGBoost.  
Unlike standard regression (MSE) which treats the target limits (19.6 and 100.0) as arbitrary cutoffs, the Tobit model statistically handles them as **censored data**.

**Key Features:**
*   **Custom Objective**: Maximizes Tobit Log-Likelihood (Gradient & Hessian derived mathematically).
*   **Bayes Optimal Prediction**: Converts the latent variable $z$ (raw output) into the expected value $E[y|x]$ considering the boundaries.
*   **Robust Pipeline**: Incorporates **Ridge Stacking** (Stage 1) and **Category Mean Encoding** for a strong baseline.

> **Note:** While standard MSE + Clipping often yields better raw RMSE on the Leaderboard due to the metric's nature, this approach generates a **statistically more natural distribution** at the boundaries. It serves as an excellent **diversity candidate** for your ensemble.

*Inspired by [broccoli beef's discussion](https://www.kaggle.com/competitions/playground-series-s6e1/discussion/667296).*

## 1. Setup & Configuration

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")

class CFG:
    SEED = 42
    N_FOLDS = 10
    TRAIN_PATH = "/kaggle/input/playground-series-s6e1/train.csv"
    TEST_PATH = "/kaggle/input/playground-series-s6e1/test.csv"
    SUB_PATH = "/kaggle/input/playground-series-s6e1/sample_submission.csv"
    TARGET = "exam_score"
    ID_COL = "id"
    
    # Tobit Parameters (Double Censoring)
    Y_L = 19.6   # Lower bound (Min score)
    Y_H = 100.0  # Upper bound (Max score)
    SIGMA = 8.5  # Estimated noise standard deviation

## 2. Tobit Theory: Mathematical Core

### Custom Objective
We define the Log-Likelihood for the doubly censored model and derive the **Gradient (1st derivative)** and **Hessian (2nd derivative)** required by XGBoost.

### Bayes Optimal Prediction
The raw output of the Tobit model is a latent variable $z \in (-\infty, \infty)$. To get the final prediction, we calculate the expected value:
$$ E[y|z] = P(L) \cdot y_L + P(H) \cdot y_H + P(Obs) \cdot E[y_{obs}] $$

In [ ]:
def bayes_optimal_pred(z_raw, ymin, ymax, sigma):
    """
    Converts latent Tobit output to expected value.
    """
    ymin_ = (ymin - z_raw) / sigma
    ymax_ = (ymax - z_raw) / sigma
    
    Phi_min = stats.norm.cdf(ymin_)
    Phi_max = stats.norm.cdf(ymax_)
    pdf_min = stats.norm.pdf(ymin_)
    pdf_max = stats.norm.pdf(ymax_)
    
    return (
        ymin * Phi_min +
        ymax * (1 - Phi_max) +
        z_raw * (Phi_max - Phi_min) -
        sigma * (pdf_max - pdf_min)
    )

def tobit_objective(y_pred, dtrain):
    """
    Custom Gradient & Hessian for Doubly Censored Tobit Likelihood
    """
    y_true = dtrain.get_label()
    sigma = CFG.SIGMA
    y_L, y_H = CFG.Y_L, CFG.Y_H
    y_pred = y_pred.flatten()

    z_L = (y_L - y_pred) / sigma
    z_H = (y_H - y_pred) / sigma
    z_y = (y_true - y_pred) / sigma
    
    Phi_L = stats.norm.cdf(z_L)
    Phi_H = stats.norm.cdf(z_H)
    phi_L = stats.norm.pdf(z_L)
    phi_H = stats.norm.pdf(z_H)
    
    # Identify censored regions (with small buffer for numerical stability)
    is_censored_low = y_true <= y_L + 1e-4
    is_censored_high = y_true >= y_H - 1e-4
    is_observed = ~(is_censored_low | is_censored_high)

    grad = np.zeros_like(y_pred)
    hess = np.ones_like(y_pred) / (sigma ** 2)
    
    # 1. Lower Censored
    if is_censored_low.any():
        denom = np.maximum(Phi_L[is_censored_low], 1e-10)
        grad[is_censored_low] = (phi_L[is_censored_low] / denom) / sigma
        term = z_L[is_censored_low] * phi_L[is_censored_low] / denom
        hess[is_censored_low] = (term + (phi_L[is_censored_low] / denom)**2) / (sigma**2)
        
    # 2. Observed
    if is_observed.any():
        grad[is_observed] = -z_y[is_observed] / sigma
        
    # 3. Upper Censored
    if is_censored_high.any():
        denom = np.maximum(1 - Phi_H[is_censored_high], 1e-10)
        grad[is_censored_high] = - (phi_H[is_censored_high] / denom) / sigma
        term = -z_H[is_censored_high] * phi_H[is_censored_high] / denom
        hess[is_censored_high] = (term + (phi_H[is_censored_high] / denom)**2) / (sigma**2)
        
    # Stability Clipping
    hess = np.clip(hess, 1e-6, 1e2)
    return grad, hess

## 3. Feature Engineering

Incorporating robust features inspired by top public kernels:
- **Magic Formula**: Linear combination known to approximate the target.
- **Polynomials & Logs**: Capturing non-linear trends.
- **Category Mean Transformer (CMT)**: Target encoding that respects ordinality (Fold-safe).

In [ ]:
class CategoryMeanTransformer:
    def __init__(self, cat_cols):
        self.cat_cols = cat_cols
        self.mappings_ = {}
    def fit(self, X, y):
        for col in self.cat_cols:
            df_temp = pd.DataFrame({col: X[col], 'target': y})
            group_means = df_temp.groupby(col)['target'].mean()
            sorted_cats = group_means.sort_values().index
            self.mappings_[col] = {cat: i for i, cat in enumerate(sorted_cats)}
        return self
    def transform(self, X):
        X_copy = X.copy()
        for col in self.cat_cols:
            X_copy[col] = X_copy[col].map(self.mappings_[col]).fillna(-1).astype(int)
        return X_copy

def fe_basic(df):
    df = df.copy()
    # Basic Expansions
    df['study_sq'] = df['study_hours'] ** 2
    df['attend_sq'] = df['class_attendance'] ** 2
    df['sleep_sq'] = df['sleep_hours'] ** 2
    df['log_study'] = np.log1p(df['study_hours'].clip(lower=0))
    
    # Efficiency & Ratios
    df['efficiency'] = (df['study_hours'] * df['class_attendance']) / (df['sleep_hours'] + 1)
    
    # Binning
    df['attend_bin'] = pd.cut(df['class_attendance'], bins=5, labels=False).astype(float)
    
    return df

## 4. Data Loading & Preparation

In [ ]:
train_df = pd.read_csv(CFG.TRAIN_PATH)
test_df = pd.read_csv(CFG.TEST_PATH)
submission_df = pd.read_csv(CFG.SUB_PATH)

CATS = train_df.select_dtypes("object").columns.tolist()
X_raw = fe_basic(train_df)
X_test_raw = fe_basic(test_df)
y = train_df[CFG.TARGET].values
num_features = [c for c in X_raw.columns if c not in CATS + [CFG.TARGET, CFG.ID_COL]]

print(f"Data Loaded. Train: {X_raw.shape}, Test: {X_test_raw.shape}")

## 5. Stage 1: RidgeCV Skeleton

Creating a strong linear meta-feature using OOF predictions.  
*Note: We apply CMT within each fold to prevent leakage.*

In [ ]:
print("Training Ridge Stage 1...")
kf = KFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
oof_ridge = np.zeros(len(X_raw))
test_ridge_accum = np.zeros(len(X_test_raw))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_raw, y), 1):
    X_tr, X_val = X_raw.iloc[train_idx], X_raw.iloc[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    # Fold-wise Encoding
    cmt = CategoryMeanTransformer(CATS).fit(X_tr, y_tr)
    scaler = StandardScaler()
    
    def transform_fold(df):
        num = df[num_features].fillna(0)
        cat = cmt.transform(df[CATS])
        return scaler.transform(pd.concat([num, cat], axis=1)) if hasattr(scaler, 'mean_') else pd.concat([num, cat], axis=1)
    
    X_tr_p = transform_fold(X_tr)
    scaler.fit(X_tr_p) # Fit scaler on train fold
    X_tr_p = scaler.transform(X_tr_p)
    X_val_p = transform_fold(X_val)
    X_test_p = transform_fold(X_test_raw)
    
    # Ridge Training
    ridge = RidgeCV(alphas=np.logspace(-3, 3, 10))
    ridge.fit(X_tr_p, y_tr)
    
    oof_ridge[val_idx] = ridge.predict(X_val_p)
    test_ridge_accum += ridge.predict(X_test_p) / CFG.N_FOLDS

print("Ridge Stage 1 Complete.")

## 6. Stage 2: XGBoost with Tobit Objective

We feed the Ridge OOF predictions into XGBoost and train using the custom `tobit_objective`.

In [ ]:
print("Starting XGBoost Tobit CV training...")

# Add Meta-Feature
drop_cols = [CFG.TARGET, CFG.ID_COL]
X_raw['ridge_pred'] = oof_ridge
X_test_raw['ridge_pred'] = test_ridge_accum

# Categorical Setup for XGB
for df in [X_raw, X_test_raw]:
    for col in CATS:
        df[col] = df[col].astype(str).astype('category')

xgb_params = {
    'learning_rate': 0.005,
    'max_depth': 9,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'tree_method': 'hist',
    'device': 'cuda',
    'enable_categorical': True,
    'random_state': CFG.SEED
}

oof_tobit_raw = np.zeros(len(X_raw))
test_tobit_accum_raw = np.zeros(len(X_test_raw))
rmse_fold_list = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_raw, y), 1):
    X_tr, X_val = X_raw.iloc[tr_idx].copy(), X_raw.iloc[val_idx].copy()
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    X_tr_final = X_tr.drop(columns=drop_cols, errors='ignore')
    X_val_final = X_val.drop(columns=drop_cols, errors='ignore')
    X_test_final = X_test_raw.drop(columns=drop_cols, errors='ignore')
    
    dtrain = xgb.DMatrix(X_tr_final, label=y_tr, enable_categorical=True)
    dval = xgb.DMatrix(X_val_final, label=y_val, enable_categorical=True)
    dtest = xgb.DMatrix(X_test_final, enable_categorical=True)
    
    # Train with Custom Objective
    model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=10000,
                      obj=tobit_objective, evals=[(dval, 'val')],
                      early_stopping_rounds=100, verbose_eval=1000)
    
    # Predict (Output is Latent Variable z)
    oof_tobit_raw[val_idx] = model.predict(dval)
    test_tobit_accum_raw += model.predict(dtest) / CFG.N_FOLDS
    
    # Evaluation (using Bayes Optimal for score check)
    oof_tobit_bayes = bayes_optimal_pred(oof_tobit_raw[val_idx], CFG.Y_L, CFG.Y_H, CFG.SIGMA)
    fold_rmse = np.sqrt(mean_squared_error(y_val, oof_tobit_bayes))
    rmse_fold_list.append(fold_rmse)
    
    print(f"Fold {fold} RMSE (Bayes): {fold_rmse:.5f}")
    gc.collect()

## 7. Bayes-Optimal Transformation & Results

The raw output from the Tobit model is the latent variable $z$.  
We convert this to the expected value using `bayes_optimal_pred` function, which statistically accounts for the censoring.

In [ ]:
# Convert Latent z -> Expected y
oof_tobit_bayes = bayes_optimal_pred(oof_tobit_raw, CFG.Y_L, CFG.Y_H, CFG.SIGMA)
test_tobit_bayes = bayes_optimal_pred(test_tobit_accum_raw, CFG.Y_L, CFG.Y_H, CFG.SIGMA)

# Clip for safety (though Bayes pred naturally stays within bounds mostly)
oof_tobit_bayes = np.clip(oof_tobit_bayes, CFG.Y_L, CFG.Y_H)
test_tobit_bayes = np.clip(test_tobit_bayes, CFG.Y_L, CFG.Y_H)

final_oof_rmse = np.sqrt(mean_squared_error(y, oof_tobit_bayes))
print(f"\n=============================================")
print(f"FINAL OOF RMSE (Bayes-optimal): {final_oof_rmse:.5f}")
print(f"=============================================")

## 8. Visualizations: The Power of Tobit

Notice how the **Residual Histogram** and **Prediction Distribution** behave.  
Tobit handles the peaks at 19.6 and 100.0 by design, creating a distinct distribution compared to standard regression.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Fold RMSE
axes[0, 0].plot(range(1, CFG.N_FOLDS+1), rmse_fold_list, marker='o', color='tab:blue')
axes[0, 0].set_xlabel("Fold")
axes[0, 0].set_ylabel("RMSE")
axes[0, 0].set_title("Fold-wise RMSE (Bayes-optimal)")

# 2. Predicted vs True
axes[0, 1].scatter(y, oof_tobit_bayes, alpha=0.1, s=1, color='tab:orange')
axes[0, 1].plot([CFG.Y_L, CFG.Y_H], [CFG.Y_L, CFG.Y_H], 'r--', label="y=x")
axes[0, 1].set_xlabel("True Score")
axes[0, 1].set_ylabel("Predicted")
axes[0, 1].set_title("OOF Prediction vs True Target")
axes[0, 1].legend()

# 3. Residuals
axes[1, 0].hist(oof_tobit_bayes - y, bins=100, color='tab:green', alpha=0.7)
axes[1, 0].set_title("Residual Distribution")
axes[1, 0].set_xlabel("Error")

# 4. Test Predictions Dist
axes[1, 1].hist(test_tobit_bayes, bins=100, color='tab:purple', alpha=0.7, label='Test Preds')
axes[1, 1].axvline(CFG.Y_L, color='red', linestyle=':', label='Min (19.6)')
axes[1, 1].axvline(CFG.Y_H, color='red', linestyle=':', label='Max (100.0)')
axes[1, 1].set_title("Test Predictions Distribution")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 9. Submission

Saving the OOF and Submission files.

In [ ]:
# Save OOF
oof_df = pd.DataFrame({
    CFG.ID_COL: train_df[CFG.ID_COL],
    CFG.TARGET: oof_tobit_bayes
})
oof_df.to_csv("oof_predictions.csv", index=False)

# Save Submission
submission_df[CFG.TARGET] = test_tobit_bayes
submission_df.to_csv("submission.csv", index=False)

print("✅ Files saved successfully.")